In [233]:
import pandas as pd
import numpy as np
import re
from math import ceil, floor
from random import randint

In [234]:
csv_data = pd.read_csv('data.csv', sep=',')

In [235]:
csv_data

,imei 1/sn,sku_name,activate_time
0,867555052196438,Redmi 9A Sea Blue 3GB RAM 32GB ROM,10/10/2020 11:10
1,862851050440414,Redmi 9A Sea Blue 2GB RAM 32GB ROM,10/10/2020 11:32
2,862851050447211,Redmi 9A Sea Blue 2GB RAM 32GB ROM,10/10/2020 11:32
3,862851050251118,Redmi 9A Sea Blue 2GB RAM 32GB ROM,10/10/2020 12:19
4,867250052667162,Redmi 9 Prime Space Blue 4GB RAM 64GB ROM,10/10/2020 12:57
...,...,...,...
813,860868043367670,Redmi 9A Nature Green 2GB RAM 32GB ROM,11/16/2020 20:37
814,860868042982750,Redmi 9A Nature Green 2GB RAM 32GB ROM,11/16/2020 20:37
815,860868045679973,Redmi 9A Nature Green 2GB RAM 32GB ROM,11/16/2020 20:41
816,860868045677134,Redmi 9A Nature Green 2GB RAM 32GB ROM,11/16/2020 20:42


In [236]:
names = pd.read_csv('names.csv',sep=',')
names

,Ghandi Goswami
0,Nerurkar Mukopadhyay
1,Taksa Kumar
2,Yateen Rabgyal
3,Manavi Rabten
4,Venu Khan
...,...
1712,Supriya Singh
1713,Jinen Rao
1714,Mitun Mukherjee
1715,Imani Sinha


In [237]:
model_keywords = ["Redmi", "9A", "3GB", "32GB", "2GB","8GB", "4GB", "64GB", "Note", "9", "Pro", "Max", "8A",
                  "Dual","Mi", "A3", "6GB", "128GB", "Prime", "9i", "8", "10T", "Y3"]
def reduced_name(name):
    gen = []
    for key in model_keywords:
        if key in name.split(" "):
            gen.append(key)
    return " ".join(gen)

In [238]:
price_dict= {
    "Redmi 9A 3GB 32GB" : 7500,
    "Redmi 9A 2GB 32GB" : 7000,
    "Redmi 9 Prime 4GB 64GB": 10000,
    "Redmi 9 Prime 4GB 128GB": 12000,
    "Redmi Note 9 Pro Max 6GB 128GB": 18500,
    "Redmi Note 9 Pro Max 6GB 64GB": 17000,
    "Redmi 8A Dual 3GB 64GB":9000,
    "Redmi 8A Dual 3GB 32GB":8300,
    "Redmi 8A Dual 2GB 32GB":7500,
    "Mi A3 4GB 64GB": 13000,
    "Redmi 9 4GB 64GB": 9000,
    "Redmi Note 9 6GB 128GB": 15000,
    "Redmi 9i 4GB 128GB": 9300,
    "Redmi 9i 4GB 64GB": 8300,
    "Redmi Note 9 4GB 128GB": 13500,
    "Redmi Note 9 4GB 64GB": 12000,
    "Redmi Note 9 Pro 4GB 64GB": 14000,
    "Redmi Note 9 Pro 4GB 128GB": 16000,
    "Redmi Note 8 4GB 64GB": 12500,
    "Mi 10T Pro 8GB 128GB": 40000,
    "Redmi Y3 4GB 64GB": 12000,
    "Redmi 9 4GB 128GB": 10000,
}

def get_price(model):
    for model_dict_single in price_dict.keys():
        if set(model.split(" ")) == set(model_dict_single.split(" ")):
            return price_dict[model_dict_single]

def get_roundoff(num):
    ceiled = int(num)
    oneth = int(str(ceiled)[-1])
    if oneth == 5 or oneth == 0:
        return ceiled
    factor = 10 - oneth if oneth != 0 and oneth != 5 and oneth > 5 else (-1 * oneth)
    return ceiled + factor

def serial_no(start, stop):
    return randint(start*10000, stop *10000)

In [244]:
from datetime import datetime
# noned = []
class Voucher:
    def __init__(self, name, array, index):
        self.index = index
        self.serial = serial_no(5,8)
        self.name = name
        self.imei = array[0]
        self.model = reduced_name(array[1])
        self.price = get_price(self.model)
        self.cgst =  self.price * 9/100
        self.sgst =  self.price * 9/100
        self.total_price = self.price + self.cgst + self.sgst
        self.roundoff = float(get_roundoff(self.total_price)) - self.total_price
        self.time = datetime.strptime(array[2], "%m/%d/%Y %H:%M")
    def getArray(self):
        return [self.serial,self.name, self.imei, self.model, self.time.strftime("%d-%m-%Y"), self.price, self.cgst, self.sgst, self.roundoff, self.total_price]

In [245]:
voucher_list =[]
names_values = names.values
for index,data in enumerate(csv_data.values):
    voucher_list.append(Voucher(names_values[index][0], data, index))

In [246]:
voucher_data = [voucher.getArray() for voucher in voucher_list]

In [247]:
frame = pd.DataFrame(voucher_data, columns=['Serial','Name', 'IMEI', 'Item', 'Time', 'Price','CGST', 'SGST','Round Off', 'Total'])

In [249]:
frame.to_csv('bills.csv')